In [ ]:
from google.cloud import bigquery

In [8]:
PROJECT_ID   = "bq-ai-hackaton"
LOCATION     = "us-central1"
GEMINI_MODEL = "gemini-2.5-pro"

In [4]:
client = bigquery.Client(project=PROJECT_ID, location=LOCATION)

In [4]:
VIDEO_ANALYSIS_PROMPT = """
# ROLE AND GOAL
You are a state-of-the-art AI visual analysis system with an expert specialization in human identification within video footage.
Your primary mission is to analyze the provided video for a critical missing person case with the highest degree of accuracy and diligence.
You must be methodical and detail-oriented in your analysis and reporting.

# TASK CONTEXT
This is a high-priority, time-sensitive analysis.
The provided video is a security footage from the street.
The objective is to determine if the missing person is visible in this video, and if so, to extract all relevant information about their presence.

# MISSING PERSON DATA
Carefully analyze the following description of the missing person. Every detail is crucial.

- **Gender:** `{gender}`
- **Approximate Age:** `{age}`
- **Build / Height:** `{build_height}`
- **Hair Color and Style:** `{hair}`
- **Clothing (Top):** `{clothing_top}`
- **Clothing (Bottom):** `{clothing_botton}`
- **Footwear:** `{footwear}`
- **Accessories:** `{accessories}`
- **Distinguishing Features:** `{features}`

# ANALYSIS INSTRUCTIONS
You must perform the following steps in your analysis:

1.  **Full Video Scan:** Meticulously review the entire video from start to finish. Do not stop after a potential first match; the person may appear multiple times.
2.  **Feature Matching:** Compare every individual in the video against the `MISSING PERSON DATA`. Assess matches based on all available criteria: clothing, build, hair, accessories, and any visible distinguishing features or mannerisms.
3.  **Confidence Score Calculation:** Based on your feature matching, calculate a confidence score from `0.0` (no match) to `1.0` (positive identification with multiple strong corroborating features). The score must be based on the quality and quantity of matching attributes. A partial clothing match is a low score; a match on clothing, build, and a distinguishing feature is a very high score.
4.  **Justification:** You MUST provide a step-by-step justification for your confidence score. List the features that matched, the features that did not match, and any features that were ambiguous or obscured (e.g., 'Face was unclear, but clothing is a 90% match').
5.  **Contextual Analysis (If Found):** If you identify the person with a confidence score greater than `0.7`:
    -   Note the exact timestamp(s) (in `HH:MM:SS` format) of their appearance.
    -   Describe their actions and behavior (e.g., 'walking quickly', 'talking on the phone', 'sitting on a bench', 'seemed distressed').
    -   Analyze if they are with anyone else. If so, provide a detailed description of each companion (gender, estimated age, clothing, etc.).
    -   Describe their direction of travel.
"""
VIDEO_ANALYSIS_PROMPT_OUTPUT = """
# OUTPUT FORMAT
Your final output MUST be a single JSON object. Do not include any text or explanations outside of this JSON structure.

**If the person is found (confidence > 0.7):**
```json
{
  "personFound": true,
  "confidenceScore": <float between 0.7 and 1.0>,
  "matchJustification": "A detailed explanation of why the confidence score was given. List all matching and non-matching features. Example: 'Confidence is high due to a perfect match on the red hooded sweatshirt, blue jeans, and black backpack. Subject's build and hair color are also consistent. Face was partially obscured, preventing a 1.0 score.'",
  "summaryOfFindings": "A concise, human-readable summary of the events. Example: 'The missing person was spotted at 00:14:32 walking east on the station platform. They appeared to be alone and were walking at a normal pace while looking at their phone.'",
  "appearances": [
    {
      "timestampStart": "HH:MM:SS",
      "timestampEnd": "HH:MM:SS",
      "actionsAndBehavior": "Detailed description of what the subject is doing during this specific timeframe.",
      "directionOfTravel": "e.g., Northbound, Towards the exit, Away from the camera",
      "companions": [
        {
          "description": "Detailed description of companion 1: Gender, estimated age, build, hair, clothing (top, bottom, shoes), and any notable accessories or features."
        }
      ]
    }
  ]
}
```
**If the person is found (confidence > 0.7):**
```json
{
  "personFound": false,
  "confidenceScore": <float between 0.0 and 0.7>,
  "matchJustification": "Explain why the person was not found or why confidence is low. Example: 'Scanned the entire video. Several individuals matched the general build, but none matched the specific red hooded sweatshirt and white sneaker combination described. Therefore, confidence is very low.'",
  "summaryOfFindings": "A concise statement of the negative finding. Example: 'After a thorough analysis of the entire video footage, there was no individual matching the provided description of the missing person.'",
  "appearances": [],
  "companions": []
}
```
"""

In [ ]:
VIDEO_ANALYSIS_PROMPT = VIDEO_ANALYSIS_PROMPT.format(
gender            ="Female",
age               ="35",
build_height      ="1.7m, 80kg",
hair              ="Black hair, sporty",
clothing_top      ="Blue shirt",
clothing_botton   ="Blue pants",
footwear          ="White snikers",
accessories       ="Dark glasses",
features          ="None"
)


In [6]:
VIDEO_ANALYSIS_PROMPT = VIDEO_ANALYSIS_PROMPT + "\n" +VIDEO_ANALYSIS_PROMPT_OUTPUT

In [10]:
VIDEO_ANALYSIS_BQ_QUERY = """
SELECT
  uri,
  AI.GENERATE(({VIDEO_ANALYSIS_PROMPT}, '\nRecording:  ', OBJ.GET_ACCESS_URL(ref, 'r')),
    connection_id => 'bq-ai-hackaton.us-central1.homeward_gcp_connection',
    endpoint => 'gemini-2.5-pro',
    model_params => JSON '{"generation_config": {"temperature": 0}}').result
FROM `homeward.video_objects`;
""".replace("{VIDEO_ANALYSIS_PROMPT}",VIDEO_ANALYSIS_PROMPT)

In [12]:
VIDEO_ANALYSIS_BQ_QUERY

'\nSELECT\n  uri,\n  AI.GENERATE((\n# ROLE AND GOAL\nYou are a state-of-the-art AI visual analysis system with an expert specialization in human identification within video footage.\nYour primary mission is to analyze the provided video for a critical missing person case with the highest degree of accuracy and diligence.\nYou must be methodical and detail-oriented in your analysis and reporting.\n\n# TASK CONTEXT\nThis is a high-priority, time-sensitive analysis.\nThe provided video is a security footage from the street.\nThe objective is to determine if the missing person is visible in this video, and if so, to extract all relevant information about their presence.\n\n# MISSING PERSON DATA\nCarefully analyze the following description of the missing person. Every detail is crucial.\n\n- **Gender:** `Female`\n- **Approximate Age:** `35`\n- **Build / Height:** `1.7m, 80kg`\n- **Hair Color and Style:** `Black hair, sporty`\n- **Clothing (Top):** `Blue shirt`\n- **Clothing (Bottom):** `Blu